In [23]:

import yfinance as yf
import time

def get_market_caps_dict(tickers, batch_size=50, delay=0.5) -> dict:
    """
    Retrieves the most recent market caps of the given tickers using yfinance.

    Args:
        tickers (list): List of ticker symbols (str).
        batch_size (int): Number of tickers to query per batch (avoid throttling).
        delay (float): Time (in seconds) to wait between batches.

    Returns:
        dict: Dictionary with tickers as keys and market cap in USD as values.
    """
    market_caps = {}

    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i+batch_size]
        data = yf.Tickers(' '.join(batch)).tickers
        for ticker in batch:
            try:
                info = data[ticker].info
                cap = info.get('marketCap', None)
                if cap is not None:
                    market_caps[ticker] = cap
            except Exception as e:
                print(f"Failed for {ticker}: {e}")


        time.sleep(delay)  # Avoid rate limits

    return market_caps

def compute_total_mcap(tickers, mcaps) -> float:
    total_mcap = 0
    for ticker in tickers:
        mcap = mcaps[ticker]
        total_mcap += mcap

    return total_mcap

def compute_weights(tickers: list, mcaps: dict) -> dict:

    total_mcap = compute_total_mcap(tickers, mcaps)

    data = {}
    for ticker in tickers:
        mcap = mcaps[ticker]
        weighted_mcap_in_idx = mcap / total_mcap
        data[ticker] = {'mcap':mcap, 'weight':weighted_mcap_in_idx}

    return data


In [24]:
tickers = ['NVDA',
           'TSM',
           'ASX',
           'AMKR',
           'MU',
           'ASML',
           'AMAT',
           'LRCX',
           'KLAC',
           'SMCI']

mcaps = get_market_caps_dict(tickers)

In [25]:
data = compute_weights(tickers, mcaps)

In [26]:
def compute_allocation(investment: float, data: dict) -> dict:

    for ticker in data:
        allocation_for_ticker = investment * data[ticker]['weight']
        data[ticker]['allocation'] = allocation_for_ticker

    return data

In [27]:
def print_data(data: dict):

    print("TICKER\tMARKETCAP\tPERCENT\tALLOCATION")
    print("____________________________________\n")
    for ticker in data:
        mcap = data[ticker]['mcap']
        percent = data[ticker]['weight']*100
        allocation = data[ticker]['allocation']
        print(f"{ticker}\t${mcap}\t{percent:.2f}\t{allocation:.2f}")

In [28]:
investment = 5000
data = compute_allocation(investment, data)
print_data(data)

TICKER	MARKETCAP	PERCENT	ALLOCATION
____________________________________

NVDA	$2876881764352	63.49	3174.38
TSM	$914440585216	20.18	1009.00
ASX	$20089661440	0.44	22.17
AMKR	$4472948736	0.10	4.94
MU	$95697518592	2.11	105.59
ASML	$280892243968	6.20	309.94
AMAT	$128524115968	2.84	141.81
LRCX	$97353826304	2.15	107.42
KLAC	$93587087360	2.07	103.26
SMCI	$19473225728	0.43	21.49
